In [49]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

### We are working with UCI Wine quality dataset

In [50]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/"

In [51]:
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [52]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [54]:
y = data.pop('quality')

X_train, X_holdout, y_train, y_holdout = train_test_split(data, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [55]:
print("X_train.size is", X_train.size)
print("X_holdout.size is", X_holdout.size)
print("y_train.size is", y_train.size)
print("y_holdout.size is", y_holdout.size)

X_train.size is 37708
X_holdout.size is 16170
y_train.size is 3428
y_holdout.size is 1470


## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [56]:
linreg = LinearRegression(fit_intercept=True, copy_X=True, n_jobs=-1, positive=False)
linreg.fit(X_train_scaled, y_train)

LinearRegression(n_jobs=-1)

In [57]:
linreg.score(X_train_scaled, y_train)

0.2904316386901199

In [58]:
linreg.score(X_holdout_scaled, y_holdout)

0.24987105940272603

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [59]:
model_predictions_train = linreg.predict(X_train_scaled)
mean_squared_error_train = mean_squared_error(y_train, model_predictions_train)
print('mean_squared_error_train is', mean_squared_error_train)

mean_squared_error_train is 0.5580606489803572


In [60]:
model_predictions_holdout = linreg.predict(X_holdout_scaled)
mean_squared_error_holdout = mean_squared_error(y_holdout, model_predictions_holdout)
print('mean_squared_error_holdout is', mean_squared_error_holdout)

mean_squared_error_holdout is 0.5842473102404546


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [61]:
coefficients = abs(linreg.coef_)
feature_names = list(data)

coef_df = pd.DataFrame(
    {'coefficients': coefficients,
     'feature_names': feature_names,
    })

In [62]:
sorted_coef_df = coef_df.sort_values(by=['coefficients'], ascending=False)
display(sorted_coef_df)

,coefficients,feature_names
7,0.665720,density
3,0.538164,residual sugar
1,0.192260,volatile acidity
8,0.150036,pH
10,0.129533,alcohol
0,0.097822,fixed acidity
9,0.062053,sulphates
5,0.042180,free sulfur dioxide
6,0.014304,total sulfur dioxide
4,0.008127,chlorides


### Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [63]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

In [64]:
lasso1.score(X_train_scaled, y_train)

0.28315074467977785

In [65]:
lasso1.score(X_holdout_scaled, y_holdout)

0.2634608744708564

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [66]:
coefficients_lasso = abs(lasso1.coef_)
feature_names_lasso = list(data)

lasso_coef_df = pd.DataFrame(
    {'coefficients': coefficients_lasso,
     'feature_names': feature_names_lasso,
    })

In [67]:
sorted_lasso_coef_df = lasso_coef_df.sort_values(by=['coefficients'], ascending=False)
display(sorted_lasso_coef_df)

,coefficients,feature_names
10,0.322425,alcohol
3,0.256363,residual sugar
7,0.235492,density
1,0.188479,volatile acidity
8,0.067277,pH
5,0.043088,free sulfur dioxide
9,0.029722,sulphates
4,0.002747,chlorides
0,0.000000,fixed acidity
2,0.000000,citric acid


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [68]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, n_jobs=-1, random_state=17)

lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, n_jobs=-1, random_state=17)

In [69]:
lasso_cv.score(X_train_scaled, y_train)

0.29041973092595263

In [70]:
lasso_cv.score(X_holdout_scaled, y_holdout)

0.25109040484686995

In [71]:
print(lasso_cv.alpha_)

0.0002833096101839324


**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [72]:
coefficients_lasso_cv = abs(lasso_cv.coef_)
feature_names_lasso_cv = list(data)

lasso_cv_coef_df = pd.DataFrame(
    {'coefficients': coefficients_lasso_cv,
     'feature_names': feature_names_lasso_cv,
    })

In [73]:
sorted_lasso_cv_coef_df = lasso_cv_coef_df.sort_values(by=['coefficients'], ascending=False)
display(sorted_lasso_cv_coef_df)

,coefficients,feature_names
7,0.648161,density
3,0.526883,residual sugar
1,0.192049,volatile acidity
8,0.146549,pH
10,0.137115,alcohol
0,0.093295,fixed acidity
9,0.060939,sulphates
5,0.042698,free sulfur dioxide
6,0.012969,total sulfur dioxide
4,0.006933,chlorides


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [74]:
lasso_model_predictions_train = lasso_cv.predict(X_train_scaled)
mean_squared_error_train = mean_squared_error(y_train, lasso_model_predictions_train)
print('mean_squared_error_train is', mean_squared_error_train)

mean_squared_error_train is 0.558070014187378


In [75]:
lasso_model_predictions_holdout = lasso_cv.predict(X_holdout_scaled)
mean_squared_error_holdout = mean_squared_error(y_holdout, lasso_model_predictions_holdout)
print('mean_squared_error_holdout is', mean_squared_error_holdout)

mean_squared_error_holdout is 0.5832976077860635


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [76]:
forest = RandomForestRegressor(random_state=17, n_jobs=-1)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(n_jobs=-1, random_state=17)

In [77]:
forest.score(X_train_scaled, y_train)

0.9331049541768827

In [78]:
forest.score(X_holdout_scaled, y_holdout)

0.5228454960179294

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [79]:
rf_model_predictions_train = forest.predict(X_train_scaled)
mean_squared_error_train = mean_squared_error(y_train, rf_model_predictions_train)
print('mean_squared_error_train is', mean_squared_error_train)

mean_squared_error_train is 0.05261155192532089


In [80]:
rf_model_predictions_holdout = forest.predict(X_holdout_scaled)
mean_squared_error_holdout = mean_squared_error(y_holdout, rf_model_predictions_holdout)
print('mean_squared_error_holdout is', mean_squared_error_holdout)

mean_squared_error_holdout is 0.37163775510204083


In [81]:
cross_val_score_calculated = np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
print('mean_squared_error_cross_val', cross_val_score_calculated)

mean_squared_error_cross_val 0.4142003732204039


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [34]:
%%time
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(), forest_params, cv=5, n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)

CPU times: total: 10.3 s
Wall time: 32min 36s


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]})

In [82]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4911529849680318)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [83]:
cross_val_score_calculated = np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_, X_train_scaled, y_train, scoring='neg_mean_squared_error')))
print('mean_squared_error_cross_val', cross_val_score_calculated)

mean_squared_error_cross_val 0.40281260277879805


In [84]:
locally_best_rf_model_predictions_train = locally_best_forest.predict(X_train_scaled)
mean_squared_error_train = mean_squared_error(y_train, locally_best_rf_model_predictions_train)
print('mean_squared_error_train is', mean_squared_error_train)

mean_squared_error_train is 0.05657046510793735


In [85]:
locally_best_rf_model_predictions_holdout = locally_best_forest.predict(X_holdout_scaled)
mean_squared_error_holdout = mean_squared_error(y_holdout, locally_best_rf_model_predictions_holdout)
print('mean_squared_error_holdout is', mean_squared_error_holdout)

mean_squared_error_holdout is 0.37006618774628003


**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [86]:
coefficients_rf_cv = abs(locally_best_forest.best_estimator_.feature_importances_)
feature_names_rf_cv = list(data)

rf_cv_coef_df = pd.DataFrame(
    {'coefficients': coefficients_rf_cv,
     'feature_names': feature_names_rf_cv,
    })

In [87]:
sorted_rf_cv_coef_df = rf_cv_coef_df.sort_values(by=['coefficients'], ascending=False)
display(sorted_rf_cv_coef_df)

,coefficients,feature_names
10,0.198946,alcohol
1,0.117351,volatile acidity
5,0.109729,free sulfur dioxide
7,0.098028,density
8,0.073304,pH
4,0.073064,chlorides
3,0.072225,residual sugar
6,0.071119,total sulfur dioxide
0,0.063982,fixed acidity
2,0.063085,citric acid


**Make conclusions about the performance of the explored 3 models in this particular prediction task.**

Three models has been used - Linear Regression, LASSO and Random Forest.\
Untuned Linear Regression showed mean_squared_error_train is 0.5580606489803572 and mean_squared_error_holdout is 0.5842473102404546\
Tuned LASSO showed mean_squared_error_train is 0.558070014187378 and mean_squared_error_holdout is 0.37163775510204083\
Untuned Random Forest showed mean_squared_error_train is 0.05261155192532089 and mean_squared_error_holdout is 0.37163775510204083\
Tuned Random Forest showed mean_squared_error_train is 0.05432684298065343 and mean_squared_error_holdout is 0.3683341989327821\
Thus, tuned **Random Forest showed the best results**.